In [1]:
import pandas as pd
import tensorflow as tf

# from notebooks.prepare.mnist_exercises import early_stopping

2025-06-02 20:11:10.596184: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-02 20:11:10.603516: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748895070.612141     632 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748895070.614653     632 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-02 20:11:10.623734: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

## Daten laden und aufbereiten
Quelle: https://www.kaggle.com/datasets/abhi8923shriv/sentiment-analysis-dataset

In [2]:
df = pd.read_csv('kaggle_sentiment/tweet_sentiment_train.csv', encoding='utf-8', encoding_errors='replace')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27481 entries, 0 to 27480
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   textID            27481 non-null  object 
 1   text              27480 non-null  object 
 2   selected_text     27480 non-null  object 
 3   sentiment         27481 non-null  object 
 4   Time of Tweet     27481 non-null  object 
 5   Age of User       27481 non-null  object 
 6   Country           27481 non-null  object 
 7   Population -2020  27481 non-null  int64  
 8   Land Area (Km�)   27481 non-null  float64
 9   Density (P/Km�)   27481 non-null  int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 2.1+ MB


In [3]:
df[:3]

,textID,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km�),Density (P/Km�)
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797,27400.0,105
2,088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044,2381740.0,18


In [4]:
import string
import re

def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    return tf.strings.regex_replace(
            stripped_html,
            '[%s]' % re.escape(string.punctuation),
            ''
            )

In [5]:
df['text'] = df['text'].astype(str)

In [6]:
sentiment_mapping = {'negative': 2, 'neutral': 0, 'positive': 1}
df['sentiment'] = df['sentiment'].replace(sentiment_mapping)
df[['text', 'sentiment']].head(10)

/tmp/ipykernel_632/1026471518.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace(sentiment_mapping)


,text,sentiment
0,"I`d have responded, if I were going",0
1,Sooo SAD I will miss you here in San Diego!!!,2
2,my boss is bullying me...,2
3,what interview! leave me alone,2
4,"Sons of ****, why couldn`t they put them on t...",2
5,http://www.dothebouncy.com/smf - some shameles...,0
6,2am feedings for the baby are fun when he is a...,1
7,Soooo high,0
8,Both of you,0
9,Journey!? Wow... u just became cooler. hehe....,1


In [7]:
texts = df['text'].values
labels = df['sentiment'].values
# Create the dataset
dataset = tf.data.Dataset.from_tensor_slices(texts, labels)

I0000 00:00:1748895072.163693     632 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9305 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070, pci bus id: 0000:01:00.0, compute capability: 8.9


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
dataset = dataset.map(lambda x, y: ((custom_standardization(x), y)))
for text, label in dataset.take(2):
    print(text.numpy())
    print(label.numpy())

In [ ]:
train_size = 22_000
val_size = 2_000
test_size = len(df) - train_size - val_size

In [ ]:
dataset = dataset.shuffle(train_size + val_size)
train_ds = dataset.take(train_size)
val_ds = dataset.skip(train_size).take(val_size)
test_ds = dataset.skip(train_size + val_size)

In [ ]:
train_ds = train_ds.batch(128).cache().prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.batch(128).cache().prefetch(tf.data.AUTOTUNE)

In [ ]:
max_sequence_length = 0
for text, label in dataset:
    if len(text.numpy()) > max_sequence_length:
        max_sequence_length = len(text.numpy())
print(max_sequence_length)

In [ ]:
import keras_hub

bert_name = "bert_tiny_en_uncased"
classifier = keras_hub.models.TextClassifier.from_preset(bert_name, sequence_lengths=256, num_classes=3)
classifier.build(input_shape=(None, 256))
classifier.summary()

In [ ]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=2,
        min_delta=0.02,
        restore_best_weights=True
        )
classifier.fit(train_ds, epochs=5, validation_data=val_ds, verbose=2)

In [ ]:
classifier.evaluate(test_ds.batch(16))

In [ ]:
import keras
import keras_hub

bert_name = "bert_tiny_en_uncased"
preprocess_layer = keras_hub.models.BertPreprocessor.from_preset(bert_name, trainable=False)
backbone = keras_hub.models.Backbone.from_preset(bert_name, trainable=False)
#tokenizer = keras_nlp.models.Tokenizer.from_preset(bert_name)

text_input = keras.Input(shape=(), dtype=tf.string, name='text')
outputs = backbone(preprocess_layer(text_input))
net = outputs['pooled_output'][:, :]
net = keras.layers.Flatten()(net)
net = keras.layers.Dropout(0.1)(net)
net = keras.layers.Dense(512, activation='relu')(net)
net = keras.layers.Dense(3, activation='softmax')(net)
model = keras.Model(text_input, net, name='bert_sentiment_model_1')
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=False)
model.compile(optimizer="adam", loss=loss_fn, metrics=['accuracy'])
model.summary()

"""
Die Ergebnisse sind nicht besonders gut. Wenn im backbone trainable=True gesetzt wird, sind die Ergebnisse deutlich besser (auch weil mehr Parameter trainiert werden können).
"""


In [ ]:
model.fit(train_ds, epochs=5, validation_data=val_ds, callbacks=[early_stopping_cb], verbose=2)